# 교차검증(Cross-Validation Test)

</br>

## 💡 교차검증이란?

- 교차검증(Cross-validation)은 머신러닝에서 모델의 일반화 능력을 평가하고 검증하기 위한 기술이다.
- 이 방법은 학습 데이터를 여러 부분집합(subsets)으로 나누고, 이 중 일부를 학습에, 나머지를 검증에 사용하는 반복적인 과정을 포함한다.

- 가장 흔히 사용되는 교차 검증 방법 중 하나는 k-겹 교차 검증(k-fold cross-validation)으로,</br>
	여기서 데이터는 'k'개의 부분집합으로 나누어지며, 각 부분집합은 순차적으로 검증 집합으로 사용된다.

- $\cal D_{train}$을 K 등분해서 각각을 K번 검증한 후 그 평균을 취한다.

	<img src="https://github.com/ElaYJ/Study_Machine_Learning/assets/153154981/8f131703-aedc-4c7a-b25e-1dd1d7fb1a51" width="57%" height="57%">

</br>

- 교차 검증의 목적은 모델이 새로운 데이터에 대해 어떻게 성능을 낼지를 평가하는 것이다.
- 이 방법은 모델이 특정 학습 데이터셋에 과적합되는 것을 방지하고, 모델의 성능을 더 신뢰성 있게 추정할 수 있게 해준다.

- 각 반복에서 모델은 서로 다른 학습 및 검증 데이터셋으로 테스트되므로,</br>
	교차 검증을 통해 얻은 성능 평가는 모델이 다양한 데이터에 얼마나 잘 일반화되는지를 더 잘 반영한다.

- 그러나 교차 검증은 추가적인 계산 비용을 필요로 하며, 특히 k가 크거나 데이터셋이 매우 큰 경우에는 시간이 오래 걸릴 수 있다.
- 또한, 데이터의 분포가 불균형하거나 특정 패턴을 따르는 경우, 교차 검증의 결과가 왜곡될 수 있다.

## 🔰 교차검증 구현

- K-Fold Cross Validation --> sklearn.model_selection.`KFold`


#### --▶ Simple Example 1.

In [9]:
import numpy as np
from sklearn.model_selection import KFold

data = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

folds = KFold(n_splits=5, shuffle=False) # 1

for train_idx, valid_idx in folds.split(data):
    print(f'훈련 데이터: {data[train_idx]}, 검증 데이터: {data[valid_idx]}')

훈련 데이터: [2 3 4 5 6 7 8 9], 검증 데이터: [0 1]
훈련 데이터: [0 1 4 5 6 7 8 9], 검증 데이터: [2 3]
훈련 데이터: [0 1 2 3 6 7 8 9], 검증 데이터: [4 5]
훈련 데이터: [0 1 2 3 4 5 8 9], 검증 데이터: [6 7]
훈련 데이터: [0 1 2 3 4 5 6 7], 검증 데이터: [8 9]


#### --▶ Simple Example 2.

In [1]:
import numpy as np
from sklearn.model_selection import KFold

X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([1, 2, 3, 4])

kf = KFold(n_splits=2)

print(kf.get_n_splits(X))
print(kf)

2
KFold(n_splits=2, random_state=None, shuffle=False)


In [2]:
X

array([[1, 2],
       [3, 4],
       [1, 2],
       [3, 4]])

In [3]:
y

array([1, 2, 3, 4])

In [4]:
for train_idx, test_idx in kf.split(X):
    print('--- idx')
    print(train_idx, test_idx)
    print('--- train data')
    print(X[train_idx])
    print('--- validation data')
    print(X[test_idx])

--- idx
[2 3] [0 1]
--- train data
[[1 2]
 [3 4]]
--- validation data
[[1 2]
 [3 4]]
--- idx
[0 1] [2 3]
--- train data
[[1 2]
 [3 4]]
--- validation data
[[1 2]
 [3 4]]


## 🔰 와인 맛 분류 데이터 ➡ KFold

In [10]:
import pandas as pd

red_wine = pd.read_csv("../dataset/winequality-red.csv", sep=';')
white_wine = pd.read_csv("../dataset/winequality-white.csv", sep=';')

red_wine['color'] = 1.
white_wine['color'] = 0.

wine = pd.concat([red_wine, white_wine])

In [11]:
wine['taste'] = [1. if grade > 5 else 0. for grade in wine['quality']]

X = wine.drop(['taste','quality'], axis=1) #--> column drop
y = wine['taste']

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

wine_tree = DecisionTreeClassifier(max_depth=2, random_state=13)
wine_tree.fit(X_train, y_train) #<--(features, labels)

y_pred_tr = wine_tree.predict(X_train)
y_pred_test = wine_tree.predict(X_test)

print('Train Acc:', accuracy_score(y_train, y_pred_tr)) #--> 훈련용 데이터의 정확도
print('Test Acc:', accuracy_score(y_test, y_pred_test)) #--> 테스트용 데이터의 정확도

Train Acc: 0.7294593034442948
Test Acc: 0.7161538461538461


#### --▶ K-Fold: 5-fold 구현

- `KFold`는 데이터를 나눈 Index를 반환한다.

- 각각의 fold에 대한 학습 후 정확도(Accuracy) 확인

In [13]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5)
wine_tree_cv = DecisionTreeClassifier(max_depth=2, random_state=13)

In [15]:
kfold.split(X)

<generator object _BaseKFold.split at 0x000002126D47D200>

In [16]:
for train_idx, test_idx in kfold.split(X):
    print(len(train_idx), len(test_idx), type(train_idx))

5197 1300 <class 'numpy.ndarray'>
5197 1300 <class 'numpy.ndarray'>
5198 1299 <class 'numpy.ndarray'>
5198 1299 <class 'numpy.ndarray'>
5198 1299 <class 'numpy.ndarray'>


In [19]:
cv_accuracy = []

for train_idx, test_idx in kfold.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    wine_tree_cv.fit(X_train, y_train)
    pred = wine_tree_cv.predict(X_test)
    print(pred)
    cv_accuracy.append(accuracy_score(y_test, pred))

cv_accuracy

[0. 0. 0. ... 1. 0. 0.]
[1. 1. 1. ... 0. 1. 1.]
[0. 0. 1. ... 0. 1. 1.]
[1. 1. 1. ... 1. 1. 1.]
[0. 1. 1. ... 1. 1. 1.]


[0.6007692307692307,
 0.6884615384615385,
 0.7090069284064665,
 0.7628945342571208,
 0.7867590454195535]

#### --▶ 5-fold의 Acc 평균

- 각 Accuracy의 분산이 크지 않다면 평균을 대표값으로 한다.

In [20]:
np.mean(cv_accuracy)

0.709578255462782

## 🔰 와인 맛 분류 데이터 ➡ Stratified KFold

#### --▶ Stratified K-Fold: 5-fold 구현

In [24]:
from sklearn.model_selection import StratifiedKFold

skfold = StratifiedKFold(n_splits=5)
wine_tree_cv = DecisionTreeClassifier(max_depth=2, random_state=13)

cv_accuracy = []

for train_idx, test_idx in skfold.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    wine_tree_cv.fit(X_train, y_train)
    pred = wine_tree_cv.predict(X_test)
    print(pred)
    cv_accuracy.append(accuracy_score(y_test, pred))

cv_accuracy

[0. 0. 0. ... 0. 1. 1.]
[1. 0. 1. ... 1. 1. 1.]
[0. 0. 1. ... 1. 1. 1.]
[0. 1. 1. ... 1. 1. 1.]
[1. 1. 0. ... 1. 1. 1.]


[0.5523076923076923,
 0.6884615384615385,
 0.7143956889915319,
 0.7321016166281755,
 0.7567359507313318]

#### --▶ Stratified 5-fold의 Acc 평균

- Accuracy의 평균이 KFold보다 더 나쁘다.

In [25]:
np.mean(cv_accuracy)

0.6888004974240539

</br>

## 🔰 `cross_val_score`

- 교차검증을 보다 가편하게 실행해주는 함수이다.

In [26]:
from sklearn.model_selection import cross_val_score

skfold = StratifiedKFold(n_splits=5)
wine_tree_cv = DecisionTreeClassifier(max_depth=2, random_state=13)

cross_val_score(wine_tree_cv, X, y, scoring=None, cv=skfold)

array([0.55230769, 0.68846154, 0.71439569, 0.73210162, 0.75673595])

In [27]:
from sklearn.model_selection import cross_val_score

skfold = StratifiedKFold(n_splits=5)
wine_tree_cv = DecisionTreeClassifier(max_depth=5, random_state=13)

cross_val_score(wine_tree_cv, X, y, scoring=None, cv=skfold)
#--> depth가 높다고 무조건 acc가 좋아지는 것도 아니다.

array([0.50076923, 0.62615385, 0.69745958, 0.7582756 , 0.74903772])

In [31]:
def skfold_depth(depth):
	import numpy as np
	from sklearn.model_selection import cross_val_score

	skfold = StratifiedKFold(n_splits=5)
	wine_tree_cv = DecisionTreeClassifier(max_depth=depth, random_state=13)

	cv_score = cross_val_score(wine_tree_cv, X, y, scoring=None, cv=skfold)
	cv_avg = np.mean(cv_score)

	print(cv_score, "--> average:", cv_avg)

In [33]:
skfold_depth(2)

[0.55230769 0.68846154 0.71439569 0.73210162 0.75673595] --> average: 0.6888004974240539


In [32]:
skfold_depth(3)

[0.56846154 0.68846154 0.71439569 0.73210162 0.75673595] --> average: 0.6920312666548233


In [34]:
skfold_depth(5)

[0.50076923 0.62615385 0.69745958 0.7582756  0.74903772] --> average: 0.6663391958311127


👉 depth가 높다고 accuracy가 마냥 좋아지는 것도 아니다.!!

</br>

## 🔰 `cross_validate`

- train score와 함께 보고 싶을 때 사용할 수 있는 함수이다.

In [35]:
from sklearn.model_selection import cross_validate

cross_validate(wine_tree_cv, X, y, scoring=None, cv=skfold, return_train_score=True)

{'fit_time': array([0.0279274 , 0.03194451, 0.03193855, 0.02194929, 0.02889085]),
 'score_time': array([0.006984  , 0.00496078, 0.00199699, 0.00299072, 0.00299215]),
 'test_score': array([0.50076923, 0.62615385, 0.69745958, 0.7582756 , 0.74903772]),
 'train_score': array([0.78795459, 0.78045026, 0.77568295, 0.76356291, 0.76279338])}

👉 과적합 현상이 나타나 있다.

- 교차검증한 train_score가 test_score보다 더 높게 나타나고 있으므로 과적합 모델임을 알 수 있다.